In [24]:
####### [DATASET]
##


####### [ADMIN] 
## sales forecasting using arima
## edit apriori parameters
## view stats regarding most popular items

############## MARKET BASKET ANALYSIS ##########

In [33]:
import pandas as pd
df = pd.read_csv('data2.csv', encoding="ISO-8859-1")
df.sample(10)
#df.shape ## 541k orders with 8 column data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
6,18,21754,LAMP,3,12/1/2010 8:34,5.95,6,Norway
8,20,21777,LACOSTE SHOES,4,12/1/2010 8:34,7.95,6,Norway
10,22,22960,UNDERWEAR RED,6,12/1/2010 8:34,4.25,7,Norway
9,21,48187,LACOSTE BLOUSE,4,12/1/2010 8:34,7.95,6,Norway
4,16,22623,LAMP RED,3,12/1/2010 8:34,4.95,6,Norway
0,1,85123A,ADIDAS STAN SMITH,6,12/1/2010 8:26,2.55,1,Norway
5,17,22622,LAMP BLUE,2,12/1/2010 8:34,9.95,6,Norway
1,2,71053,NIKE FORERUNNER,6,12/1/2010 8:26,3.39,2,Norway
3,15,84969,LAMP RED,6,12/1/2010 8:34,4.25,6,Norway
2,3,84406B,ADIDAS STAN SMITH,8,12/1/2010 8:26,2.75,1,Norway


In [34]:
## preprocess
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['CustomerID'], inplace=True)
df['CustomerID'] = df['CustomerID'].astype('str')

## get data only from France to reduce database bcz of computations
## Create a pivot table with relationship btwn invoiceno and description
basket = df.pivot_table(index="CustomerID", columns="Description", values="Quantity").fillna(0)
basket ## it has about 1.5k items

Description,ADIDAS STAN SMITH,LACOSTE BLOUSE,LACOSTE SHOES,LACOSTE SUNGLASSES,LAMP,LAMP BLUE,LAMP RED,NIKE FORERUNNER,SOCKS BLUE,UNDERWEAR RED
CustomerID,,,,,,,,,,
1,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0
6,0.0,4.0,4.0,3.0,3.0,2.0,4.5,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,6.0


In [37]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units) 
basket.shape

(4, 10)

In [42]:
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules

frequent_itemsets = apriori(basket_sets, min_support=0.05, use_colnames=True)

In [98]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(LACOSTE BLOUSE),(LACOSTE SHOES),0.25,0.25,0.25,1.0,4.0,0.1875,inf
1,(LACOSTE SHOES),(LACOSTE BLOUSE),0.25,0.25,0.25,1.0,4.0,0.1875,inf
2,(LACOSTE BLOUSE),(LACOSTE SUNGLASSES),0.25,0.25,0.25,1.0,4.0,0.1875,inf
3,(LACOSTE SUNGLASSES),(LACOSTE BLOUSE),0.25,0.25,0.25,1.0,4.0,0.1875,inf
4,(LACOSTE BLOUSE),(LAMP),0.25,0.25,0.25,1.0,4.0,0.1875,inf
...,...,...,...,...,...,...,...,...,...
599,(LAMP),"(LAMP RED, LACOSTE SHOES, LAMP BLUE, LACOSTE B...",0.25,0.25,0.25,1.0,4.0,0.1875,inf
600,(LACOSTE SHOES),"(LAMP RED, LAMP, LAMP BLUE, LACOSTE BLOUSE, LA...",0.25,0.25,0.25,1.0,4.0,0.1875,inf
601,(LAMP BLUE),"(LAMP RED, LAMP, LACOSTE SHOES, LACOSTE BLOUSE...",0.25,0.25,0.25,1.0,4.0,0.1875,inf
602,(LACOSTE BLOUSE),"(LAMP RED, LAMP, LACOSTE SHOES, LAMP BLUE, LAC...",0.25,0.25,0.25,1.0,4.0,0.1875,inf


In [99]:
rules = rules[ (rules['lift'] >= 3) &
       (rules['confidence'] >= 0.8) ]

In [100]:
## !! Good rules will have a lift >1.

In [141]:
rules['antecedents'] = rules[['antecedents']].applymap(lambda x: f'{x}')
#rules['consequents'] = rules['consequents'].applymap(lambda x: list(x)) ## this are going to be suggestions based on antecedent
# rules.sample(100)

def recommend_in_cart(items: list):
    recommendations = rules.loc[rules['antecedents'].str.contains(items[0]) & rules['antecedents'].str.contains(items[1])]
    return recommendations

rec = recommend_in_cart(['LAMP', 'LAMP BLUE'])
rec


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
6,['LAMP BLUE'],[LACOSTE BLOUSE],0.25,0.25,0.25,1.0,4.0,0.1875,inf
14,['LAMP BLUE'],[LACOSTE SHOES],0.25,0.25,0.25,1.0,4.0,0.1875,inf
20,['LAMP BLUE'],[LACOSTE SUNGLASSES],0.25,0.25,0.25,1.0,4.0,0.1875,inf
24,['LAMP BLUE'],[LAMP],0.25,0.25,0.25,1.0,4.0,0.1875,inf
28,['LAMP BLUE'],[LAMP RED],0.25,0.25,0.25,1.0,4.0,0.1875,inf
...,...,...,...,...,...,...,...,...,...
589,"['LAMP BLUE', 'LAMP']","[LACOSTE BLOUSE, LAMP RED, LACOSTE SUNGLASSES,...",0.25,0.25,0.25,1.0,4.0,0.1875,inf
592,"['LAMP BLUE', 'LACOSTE SHOES']","[LACOSTE BLOUSE, LAMP RED, LAMP, LACOSTE SUNGL...",0.25,0.25,0.25,1.0,4.0,0.1875,inf
595,"['LAMP BLUE', 'LACOSTE BLOUSE']","[LAMP RED, LAMP, LACOSTE SUNGLASSES, LACOSTE S...",0.25,0.25,0.25,1.0,4.0,0.1875,inf
596,"['LAMP BLUE', 'LACOSTE SUNGLASSES']","[LACOSTE BLOUSE, LAMP RED, LAMP, LACOSTE SHOES]",0.25,0.25,0.25,1.0,4.0,0.1875,inf
